In [1]:
import os

import numpy as np
import cv2 as cv

try:
    from sklearnex import patch_sklearn
    patch_sklearn()
except:
    pass

from sklearn.feature_selection import r_regression, chi2, f_classif, SelectKBest
from sklearn.linear_model import LogisticRegression

from skimage.feature import hog, daisy

try:
    %load_ext autotime
except:
    pass

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
2023-06-04 17:22:26.516215: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-04 17:22:26.560519: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 17:22:27.163655: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


time: 1.32 s (started: 2023-06-04 17:22:26 +08:00)


In [2]:
if os.path.isfile("cat_daisy.npy") and os.path.isfile("dog_daisy.npy"):
    cat = np.load("cat_daisy.npy")
    dog = np.load("dog_daisy.npy")
else:
    from joblib import Parallel, delayed
    def f(x: bool, i: int):
        return daisy(cv.resize(cv.cvtColor(cv.imread(f"train/{('cat', 'dog')[x]}.{i}.jpg"), cv.COLOR_BGR2GRAY), (256, 256)),
                     step=16, radius=64).ravel()
    cat = np.array(Parallel(n_jobs=-1)(delayed(f)(False, i) for i in range(N)))
    dog = np.array(Parallel(n_jobs=-1)(delayed(f)(True, i) for i in range(N)))
    np.save("cat_daisy.npy", cat)
    np.save("dog_daisy.npy", dog)

time: 4min 55s (started: 2023-06-04 16:29:30 +08:00)


In [2]:
N = 12500
N_TRAIN = 10000
N_VAL = 1000
N_TEST = 1500

y_train = np.concatenate([np.zeros((N_TRAIN, 1, 1), dtype=int), np.ones((N_TRAIN, 1, 1), dtype=int)])
y_val = np.concatenate([np.zeros((N_VAL, 1, 1), dtype=int), np.ones((N_VAL, 1, 1), dtype=int)])
y_test = np.concatenate([np.zeros((N_TEST, 1, 1), dtype=int), np.ones((N_TEST, 1, 1), dtype=int)])

cat = np.reshape(np.load("cat_daisy.npy"), (N, 1, 12800))
dog = np.reshape(np.load("dog_daisy.npy"), (N, 1, 12800))

x_train = np.concatenate([cat[:N_TRAIN], dog[:N_TRAIN]])
x_val = np.concatenate([cat[N_TRAIN:N_TRAIN+N_VAL], dog[N_TRAIN:N_TRAIN+N_VAL]])
x_test = np.concatenate([cat[-N_TEST:], dog[-N_TEST:]])

time: 2.14 s (started: 2023-06-04 17:22:33 +08:00)


In [3]:
model = Sequential([
    InputLayer(input_shape=(1, 12800)),
    # Conv1D(1024, 5, padding='same', activation='relu'),
    # Conv1D(512, 11, padding='same', activation='relu'),
    # Conv1D(256, 17, padding='same', activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.optimizers.Adam(0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

model.summary()

model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=200, batch_size=100,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

print(model.evaluate(x_test, y_test, batch_size=100, return_dict=True))

model.save("model_daisy")

del model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 1)              12801     
                                                                 
Total params: 12,801
Trainable params: 12,801
Non-trainable params: 0
_________________________________________________________________


2023-06-04 17:22:49.732626: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/200
200/200 [==============================] - 1s 4ms/step - loss: 0.6764 - binary_accuracy: 0.5842 - precision: 0.5844 - recall: 0.5833 - val_loss: 0.6609 - val_binary_accuracy: 0.6225 - val_precision: 0.6457 - val_recall: 0.5430
Epoch 2/200
200/200 [==============================] - 0s 2ms/step - loss: 0.6588 - binary_accuracy: 0.6169 - precision: 0.6218 - recall: 0.5965 - val_loss: 0.6486 - val_binary_accuracy: 0.6405 - val_precision: 0.6253 - val_recall: 0.7010
Epoch 3/200
200/200 [==============================] - 0s 2ms/step - loss: 0.6475 - binary_accuracy: 0.6351 - precision: 0.6381 - recall: 0.6242 - val_loss: 0.6414 - val_binary_accuracy: 0.6465 - val_precision: 0.6194 - val_recall: 0.7600
Epoch 4/200
200/200 [==============================] - 0s 2ms/step - loss: 0.6390 - binary_accuracy: 0.6470 - precision: 0.6482 - recall: 0.6429 - val_loss: 0.6322 - val_binary_accuracy: 0.6475 - val_precision: 0.6494 - val_recall: 0.6410
Epoch 5/200
200/200 [=======================

2023-06-04 17:24:37.584907: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,1,12800]
	 [[{{node input_1}}]]
2023-06-04 17:24:37.593770: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,1,12800]
	 [[{{node input_1}}]]
2023-06-04 17:24:37.603372: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1,12800]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: model_daisy/assets


2023-06-04 17:24:37.904196: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_input_1' with dtype float and shape [?,1,12800]
	 [[{{node serving_default_input_1}}]]
INFO:tensorflow:Assets written to: model_daisy/assets


time: 1min 51s (started: 2023-06-04 17:22:46 +08:00)


In [4]:
pearson = SelectKBest(r_regression, k=128*87)
pearson.fit(np.reshape(x_train, (N_TRAIN << 1, 12800)), np.reshape(y_train, (N_TRAIN << 1,)))

model = Sequential([
    InputLayer(input_shape=(1, 128*87)),
    # Conv1D(1024, 5, padding='same', activation='relu'),
    # Conv1D(512, 11, padding='same', activation='relu'),
    # Conv1D(256, 17, padding='same', activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.optimizers.Adam(0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

model.summary()

model.fit(np.reshape(pearson.transform(np.reshape(x_train, (N_TRAIN << 1, 12800))), (N_TRAIN << 1, 1, 128*87)), y_train,
          validation_data=(np.reshape(pearson.transform(np.reshape(x_val, (N_VAL << 1, 12800))), (N_VAL << 1, 1, 128*87)), y_val), epochs=200, batch_size=100,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

print(model.evaluate(np.reshape(pearson.transform(np.reshape(x_test, (N_TEST << 1, 12800))), (N_TEST << 1, 1, 128*87)), y_test, batch_size=100, return_dict=True))

model.save("model_daisy_pearson")

del pearson, model

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 1, 1)              11137     
                                                                 
Total params: 11,137
Trainable params: 11,137
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
200/200 [==============================] - 1s 4ms/step - loss: 0.6809 - binary_accuracy: 0.5778 - precision_1: 0.5744 - recall_1: 0.6009 - val_loss: 0.6684 - val_binary_accuracy: 0.6120 - val_precision_1: 0.6672 - val_recall_1: 0.4470
Epoch 2/200
200/200 [==============================] - 0s 2ms/step - loss: 0.6639 - binary_accuracy: 0.6199 - precision_1: 0.6249 - recall_1: 0.6000 - val_loss: 0.6546 - val_binary_accuracy: 0.6335 - val_precision_1: 0.6287 - val_recall_1: 0.6520
Epoch 3/200
200/200 [==============================] - 0s 2ms/step - loss: 0.6531 - 

2023-06-04 17:26:59.491622: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_2' with dtype float and shape [?,1,11136]
	 [[{{node input_2}}]]
2023-06-04 17:26:59.496971: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_2' with dtype float and shape [?,1,11136]
	 [[{{node input_2}}]]
2023-06-04 17:26:59.502767: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1,11136]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: model_daisy_pearson/assets


INFO:tensorflow:Assets written to: model_daisy_pearson/assets


time: 1min 48s (started: 2023-06-04 17:25:10 +08:00)


In [5]:
chi_square = SelectKBest(chi2, k=128*87)
chi_square.fit(np.reshape(x_train, (N_TRAIN << 1, 12800)), np.reshape(y_train, (N_TRAIN << 1,)))

model = Sequential([
    InputLayer(input_shape=(1, 128*87)),
    # Conv1D(1024, 5, padding='same', activation='relu'),
    # Conv1D(512, 11, padding='same', activation='relu'),
    # Conv1D(256, 17, padding='same', activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.optimizers.Adam(0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

model.summary()

model.fit(np.reshape(chi_square.transform(np.reshape(x_train, (N_TRAIN << 1, 12800))), (N_TRAIN << 1, 1, 128*87)), y_train,
          validation_data=(np.reshape(chi_square.transform(np.reshape(x_val, (N_VAL << 1, 12800))), (N_VAL << 1, 1, 128*87)), y_val), epochs=200, batch_size=100,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

print(model.evaluate(np.reshape(chi_square.transform(np.reshape(x_test, (N_TEST << 1, 12800))), (N_TEST << 1, 1, 128*87)), y_test, batch_size=100, return_dict=True))

model.save("model_daisy_chi_square")

del chi_square, model

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 1, 1)              11137     
                                                                 
Total params: 11,137
Trainable params: 11,137
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
200/200 [==============================] - 1s 4ms/step - loss: 0.6785 - binary_accuracy: 0.5944 - precision_2: 0.5957 - recall_2: 0.5878 - val_loss: 0.6634 - val_binary_accuracy: 0.6310 - val_precision_2: 0.6307 - val_recall_2: 0.6320
Epoch 2/200
200/200 [==============================] - 0s 2ms/step - loss: 0.6602 - binary_accuracy: 0.6186 - precision_2: 0.6234 - recall_2: 0.5992 - val_loss: 0.6505 - val_binary_accuracy: 0.6375 - val_precision_2: 0.6286 - val_recall_2: 0.6720
Epoch 3/200
200/200 [==============================] - 0s 2ms/step - loss: 0.6497 - 

2023-06-04 17:29:24.037027: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_3' with dtype float and shape [?,1,11136]
	 [[{{node input_3}}]]
2023-06-04 17:29:24.042286: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_3' with dtype float and shape [?,1,11136]
	 [[{{node input_3}}]]
2023-06-04 17:29:24.048058: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1,11136]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: model_daisy_chi_square/assets


INFO:tensorflow:Assets written to: model_daisy_chi_square/assets


time: 1min 45s (started: 2023-06-04 17:27:38 +08:00)


In [6]:
anova = SelectKBest(f_classif, k=128*87)
anova.fit(np.reshape(x_train, (N_TRAIN << 1, 12800)), np.reshape(y_train, (N_TRAIN << 1,)))

model = Sequential([
    InputLayer(input_shape=(1, 128*87)),
    # Conv1D(1024, 5, padding='same', activation='relu'),
    # Conv1D(512, 11, padding='same', activation='relu'),
    # Conv1D(256, 17, padding='same', activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.optimizers.Adam(0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

model.summary()

model.fit(np.reshape(anova.transform(np.reshape(x_train, (N_TRAIN << 1, 12800))), (N_TRAIN << 1, 1, 128*87)), y_train,
          validation_data=(np.reshape(anova.transform(np.reshape(x_val, (N_VAL << 1, 12800))), (N_VAL << 1, 1, 128*87)), y_val), epochs=200, batch_size=100,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

print(model.evaluate(np.reshape(anova.transform(np.reshape(x_test, (N_TEST << 1, 12800))), (N_TEST << 1, 1, 128*87)), y_test, batch_size=100, return_dict=True))

model.save("model_daisy_anova")

del anova, model

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 1, 1)              11137     
                                                                 
Total params: 11,137
Trainable params: 11,137
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
200/200 [==============================] - 1s 4ms/step - loss: 0.6784 - binary_accuracy: 0.5838 - precision_3: 0.5866 - recall_3: 0.5678 - val_loss: 0.6632 - val_binary_accuracy: 0.6205 - val_precision_3: 0.6535 - val_recall_3: 0.5130
Epoch 2/200
200/200 [==============================] - 1s 3ms/step - loss: 0.6600 - binary_accuracy: 0.6199 - precision_3: 0.6218 - recall_3: 0.6123 - val_loss: 0.6500 - val_binary_accuracy: 0.6290 - val_precision_3: 0.6466 - val_recall_3: 0.5690
Epoch 3/200
200/200 [==============================] - 1s 3ms/step - loss: 0.6502 - 

2023-06-04 17:31:37.119398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_4' with dtype float and shape [?,1,11136]
	 [[{{node input_4}}]]
2023-06-04 17:31:37.127327: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_4' with dtype float and shape [?,1,11136]
	 [[{{node input_4}}]]
2023-06-04 17:31:37.136459: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1,11136]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: model_daisy_anova/assets


INFO:tensorflow:Assets written to: model_daisy_anova/assets


time: 1min 52s (started: 2023-06-04 17:29:45 +08:00)
